In [35]:
from trulens_eval import Tru
from trulens_eval.feedback import Groundedness
from trulens_eval import TruLlama
from trulens_eval import OpenAI as fOpenAI
from trulens_eval import Feedback

from llama_index import Document
from llama_index import SimpleDirectoryReader
from llama_index import ServiceContext, VectorStoreIndex, StorageContext
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index import load_index_from_storage
from llama_index.llms import OpenAI

import openai

import numpy as np
import pandas as pd

import os
from dotenv import load_dotenv, find_dotenv

In [36]:
print(os.getcwd())

c:\Users\marlo\OneDrive\Desktop\Anaconda\Fun\Deep_Learning\Semantic_Search_Project\llama_index


In [37]:
# sys.path.append('../..')
dotenv_path = 'KEYs.env'  
_ = load_dotenv(dotenv_path)

openai.api_key = os.environ['OPENAI_API_KEY']

In [38]:
def build_sentence_window_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=5,
    save_dir="sentence_index",
):
    # create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentence_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
    )
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex.from_documents(
            documents, service_context=sentence_context
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
        )

    return sentence_index


def get_sentence_window_query_engine(
    sentence_index, similarity_top_k=6, rerank_top_n=3
):
    # define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
    )
    return sentence_window_engine

In [39]:
documents = SimpleDirectoryReader(input_dir="C:/Users/marlo/OneDrive/Desktop/Anaconda/Fun/Deep_Learning/Semantic_Search_Project/Database", 
                                  recursive=True, 
                                  num_files_limit=10).load_data()

# SimpleDirectoryReader(
#     input_dir="path/to/directory", exclude=["path/to/file1", "path/to/file2"]
# )

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [42]:
count = 0
for doc in documents:
    count +=1
print("doc.text:",doc.text)
print("doc:",doc)
print(count)

doc.text:  
 
 
___________________________________________________  
 Verabschiedun g durch die Leitlinie durch die K onsensuskonferenz am 24.5.2018 
in Hannover  
 
Annahme der aktuellen Version durch Mitgliederbeschluss  
der Deutschen AIDS Gesellschaft e.V.  (DAIG) am 22.6.2018 in K öln 
 
 
Leitlinienkoordination:  
Prof. Dr. med. Hans-Jürgen Ste llbrink 
Grindelallee 35  
20146 Ham burg 
 
 
 
Erstveröffentlichung:       05/2018  
 
Überarbeitung von:        
 
Nächste Überprüfung geplant:     05/2023  
 
 
 
 
Die AWMF erfasst und publiziert die Leitlinien der Fachgesellschaften mit 
größtmöglicher Sorgfalt -  dennoch kann die AWMF für die Richtigkeit des 
Inhalts keine Verantwortung übernehmen. Insbesondere bei 
Dosierungsangaben sind stets die Angaben der Herstell er zu beachten!  
 
Autorisiert für elektronische Publikation: AWMF online  
 
 
doc: Doc ID: 66b734b0-3e1e-4f92-a3fe-10af970cf4b2
Text: ___________________________________________________
Verabschiedun g durch die L

In [12]:
# index = build_sentence_window_index(
#     [document],
#     llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
#     save_dir="./sentence_index",
# )

In [13]:
query_engine = get_sentence_window_query_engine(index, similarity_top_k=6)

In [14]:
question = 'Ist eine Anamnese auch genannt als Test in den Dokumenten?'

response = query_engine.query(question)

In [15]:
print(response)

Nein, eine Anamnese wird in den Dokumenten nicht als Test bezeichnet.


In [19]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [20]:
print(eval_questions)

['Wann wird die bariatrische OP emopfohlen?', 'Wie untersuche ich eine Wespenstichallergie?', 'Was sind die ersten Tests um AIDS zu diagnostizieren?', 'Ist eine Anamnese auch genannt als Test in den Dokumenten?']


#### TruLens Evaluation

##### 1. Answer Relevance

In [22]:
provider = fOpenAI()

In [23]:
f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input_output()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


##### 2. Context Relevance

In [24]:
context_selection = TruLlama.select_source_nodes().node.text

In [25]:
f_qs_relevance = (
    Feedback(provider.qs_relevance,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


##### 3. Groundedness

In [26]:
provider = fOpenAI()

In [27]:
grounded = Groundedness(groundedness_provider=provider)

In [28]:
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons,
             name="Groundedness"
            )
    .on(context_selection)
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [29]:
def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)

In [30]:
sentence_window_engine = \
tru_recorder = TruLlama(
    query_engine,
    app_id="App_1",
    feedbacks=[
        f_qa_relevance,
        f_qs_relevance,
        f_groundedness
    ]
)

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [31]:
for question in eval_questions:
    with tru_recorder as recording:
        sentence_window_engine.query(question)

Function <method-wrapper '__call__' of method object at 0x0000022D5C9DA880> has not been instrumented. This may be ok if it will call a function that has been instrumented exactly once. Otherwise unexpected results may follow. You can use `AddInstruments.method` of `trulens_eval.instruments` before you use the `TruLlama` wrapper to make sure `__call__` does get instrumented. `TruLlama` method `print_instrumented` may be used to see methods that have been instrumented. 



`query` will be deprecated soon; To record results of your app's execution, use one of these options to invoke your app:
    (1) Use the `with_` method:
        ```python
        app # your app
        tru_app_recorder: TruLlama = TruLlama(app, ...)
        result = tru_app_recorder.with_(app.query, ...args/kwargs-to-app.query...)
        ```
    (2) Use TruLlama as a context manager: 
        ```python
        app # your app
        tru_app_recorder: TruLlama = TruLlama(app, ...)
        with tru_app_recorder:
            result = app.query(...args/kwargs-to-app.query...)
        
        ```


`query_with_record` will be deprecated soon; To record results of your app's execution, use one of these options to invoke your app:
    (1) Use the `with_record` method:
        ```python
        app # your app
        tru_app_recorder: TruLlama = TruLlama(app, ...)
        result, record = tru_app_recorder.with_record(app.query, ...args/kwargs-to-app.query...)
        ```
    (2) Use TruLlam

Function <method-wrapper '__call__' of method object at 0x0000022D85BF6500> has not been instrumented. This may be ok if it will call a function that has been instrumented exactly once. Otherwise unexpected results may follow. You can use `AddInstruments.method` of `trulens_eval.instruments` before you use the `TruLlama` wrapper to make sure `__call__` does get instrumented. `TruLlama` method `print_instrumented` may be used to see methods that have been instrumented. 



`query` will be deprecated soon; To record results of your app's execution, use one of these options to invoke your app:
    (1) Use the `with_` method:
        ```python
        app # your app
        tru_app_recorder: TruLlama = TruLlama(app, ...)
        result = tru_app_recorder.with_(app.query, ...args/kwargs-to-app.query...)
        ```
    (2) Use TruLlama as a context manager: 
        ```python
        app # your app
        tru_app_recorder: TruLlama = TruLlama(app, ...)
        with tru_app_recorder:
            result = app.query(...args/kwargs-to-app.query...)
        
        ```


`query_with_record` will be deprecated soon; To record results of your app's execution, use one of these options to invoke your app:
    (1) Use the `with_record` method:
        ```python
        app # your app
        tru_app_recorder: TruLlama = TruLlama(app, ...)
        result, record = tru_app_recorder.with_record(app.query, ...args/kwargs-to-app.query...)
        ```
    (2) Use TruLlam

Function <method-wrapper '__call__' of method object at 0x0000022D5E729B00> has not been instrumented. This may be ok if it will call a function that has been instrumented exactly once. Otherwise unexpected results may follow. You can use `AddInstruments.method` of `trulens_eval.instruments` before you use the `TruLlama` wrapper to make sure `__call__` does get instrumented. `TruLlama` method `print_instrumented` may be used to see methods that have been instrumented. 



`query` will be deprecated soon; To record results of your app's execution, use one of these options to invoke your app:
    (1) Use the `with_` method:
        ```python
        app # your app
        tru_app_recorder: TruLlama = TruLlama(app, ...)
        result = tru_app_recorder.with_(app.query, ...args/kwargs-to-app.query...)
        ```
    (2) Use TruLlama as a context manager: 
        ```python
        app # your app
        tru_app_recorder: TruLlama = TruLlama(app, ...)
        with tru_app_recorder:
            result = app.query(...args/kwargs-to-app.query...)
        
        ```


`query_with_record` will be deprecated soon; To record results of your app's execution, use one of these options to invoke your app:
    (1) Use the `with_record` method:
        ```python
        app # your app
        tru_app_recorder: TruLlama = TruLlama(app, ...)
        result, record = tru_app_recorder.with_record(app.query, ...args/kwargs-to-app.query...)
        ```
    (2) Use TruLlam

Function <method-wrapper '__call__' of method object at 0x0000022D5CF34E00> has not been instrumented. This may be ok if it will call a function that has been instrumented exactly once. Otherwise unexpected results may follow. You can use `AddInstruments.method` of `trulens_eval.instruments` before you use the `TruLlama` wrapper to make sure `__call__` does get instrumented. `TruLlama` method `print_instrumented` may be used to see methods that have been instrumented. 



`query` will be deprecated soon; To record results of your app's execution, use one of these options to invoke your app:
    (1) Use the `with_` method:
        ```python
        app # your app
        tru_app_recorder: TruLlama = TruLlama(app, ...)
        result = tru_app_recorder.with_(app.query, ...args/kwargs-to-app.query...)
        ```
    (2) Use TruLlama as a context manager: 
        ```python
        app # your app
        tru_app_recorder: TruLlama = TruLlama(app, ...)
        with tru_app_recorder:
            result = app.query(...args/kwargs-to-app.query...)
        
        ```


`query_with_record` will be deprecated soon; To record results of your app's execution, use one of these options to invoke your app:
    (1) Use the `with_record` method:
        ```python
        app # your app
        tru_app_recorder: TruLlama = TruLlama(app, ...)
        result, record = tru_app_recorder.with_record(app.query, ...args/kwargs-to-app.query...)
        ```
    (2) Use TruLlam

In [32]:
tru = Tru()

records, feedback = tru.get_records_and_feedback(app_ids=[])
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Context Relevance,Answer Relevance,Groundedness,Context Relevance_calls,Answer Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_2ed9c45be7f0652162fbc9e5e49428be,"""Wann wird die bariatrische OP emopfohlen?""","""Die bariatrische Operation wird empfohlen, we...",-,"{""record_id"": ""record_hash_2ed9c45be7f0652162f...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-23T09:31:14.135003"", ""...",2023-12-23T09:31:30.246954,0.4,0.9,1.0,[{'args': {'question': 'Wann wird die bariatri...,[{'args': {'prompt': 'Wann wird die bariatrisc...,[{'args': {'source': 'In Kliniken mit mehr als...,16,1799,0.002800
1,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_3035fe347e98d82303cef6577365c319,"""Wann wird die bariatrische OP emopfohlen?""","""Die bariatrische Operation wird empfohlen, we...",-,"{""record_id"": ""record_hash_3035fe347e98d82303c...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-23T09:31:14.135003"", ""...",2023-12-23T09:31:31.531660,0.4,0.9,1.0,[{'args': {'question': 'Wann wird die bariatri...,[{'args': {'prompt': 'Wann wird die bariatrisc...,[{'args': {'source': 'In Kliniken mit mehr als...,16,1799,0.002800
2,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_20cf2087031c692c15ded7c03391b498,"""Wie untersuche ich eine Wespenstichallergie?""","""Um eine Wespenstichallergie zu untersuchen, k...",-,"{""record_id"": ""record_hash_20cf2087031c692c15d...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-23T09:31:32.639123"", ""...",2023-12-23T09:31:43.558161,0.0,1.0,0.0,[{'args': {'question': 'Wie untersuche ich ein...,[{'args': {'prompt': 'Wie untersuche ich eine ...,[{'args': {'source': 'Machen Sie bitte in folg...,10,1693,0.002640
3,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_300a99e429ae3dfe7c9f73f4540701da,"""Wie untersuche ich eine Wespenstichallergie?""","""Um eine Wespenstichallergie zu untersuchen, k...",-,"{""record_id"": ""record_hash_300a99e429ae3dfe7c9...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-23T09:31:32.639123"", ""...",2023-12-23T09:31:44.279939,0.0,1.0,0.0,[{'args': {'question': 'Wie untersuche ich ein...,[{'args': {'prompt': 'Wie untersuche ich eine ...,[{'args': {'source': 'Machen Sie bitte in folg...,10,1693,0.002640
4,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_809f920b37bcbd70109c8e066a3102fb,"""Was sind die ersten Tests um AIDS zu diagnost...","""Die ersten Tests zur Diagnose von AIDS sind d...",-,"{""record_id"": ""record_hash_809f920b37bcbd70109...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-23T09:31:45.413347"", ""...",2023-12-23T09:31:55.661394,0.2,1.0,NaN,[{'args': {'question': 'Was sind die ersten Te...,[{'args': {'prompt': 'Was sind die ersten Test...,NaN,10,1727,0.002651


In [33]:
pd.set_option("display.max_colwidth", None)
records[["input", "output"] + feedback]

,input,output,Context Relevance,Answer Relevance,Groundedness
0,"""Wann wird die bariatrische OP emopfohlen?""","""Die bariatrische Operation wird empfohlen, wenn bestimmte Kontraindikationen nicht bestehen, wie instabile psychopathologische Zust\u00e4nde, konsumierende und neoplastische Erkrankungen, aktive Substanzabh\u00e4ngigkeit, unbehandelte Bulimia nervosa, chronische Erkrankungen wie Leberzirrhose oder andere schwer gesundheitlich einschr\u00e4nkende Erkrankungen, die sich durch den postoperativen katabolen Stoffwechsel verschlechtern k\u00f6nnen. Es wird auch empfohlen, dass die Operation in Kliniken mit einer h\u00f6heren Anzahl von bariatrischen Operationen durchgef\u00fchrt wird, da dies mit einer geringeren postoperativen Mortalit\u00e4t und Morbidit\u00e4t sowie einem k\u00fcrzeren station\u00e4ren Aufenthalt verbunden ist. Eine lebenslange interdisziplin\u00e4re Nachsorge wird ebenfalls empfohlen.""",0.4,0.9,1.0
1,"""Wann wird die bariatrische OP emopfohlen?""","""Die bariatrische Operation wird empfohlen, wenn bestimmte Kontraindikationen nicht bestehen, wie instabile psychopathologische Zust\u00e4nde, konsumierende und neoplastische Erkrankungen, aktive Substanzabh\u00e4ngigkeit, unbehandelte Bulimia nervosa, chronische Erkrankungen wie Leberzirrhose oder andere schwer gesundheitlich einschr\u00e4nkende Erkrankungen, die sich durch den postoperativen katabolen Stoffwechsel verschlechtern k\u00f6nnen. Es wird auch empfohlen, dass die Operation in Kliniken mit einer h\u00f6heren Anzahl von bariatrischen Operationen durchgef\u00fchrt wird, da dies mit einer geringeren postoperativen Mortalit\u00e4t und Morbidit\u00e4t sowie einem k\u00fcrzeren station\u00e4ren Aufenthalt verbunden ist. Eine lebenslange interdisziplin\u00e4re Nachsorge wird ebenfalls empfohlen.""",0.4,0.9,1.0
2,"""Wie untersuche ich eine Wespenstichallergie?""","""Um eine Wespenstichallergie zu untersuchen, k\u00f6nnen verschiedene diagnostische Verfahren eingesetzt werden. Eine M\u00f6glichkeit ist der sogenannte Hauttest, bei dem eine kleine Menge des Allergens auf die Haut aufgetragen wird und anschlie\u00dfend die Reaktion beobachtet wird. Eine andere Methode ist der Bluttest, bei dem das Blut auf spezifische Antik\u00f6rper gegen das Wespenallergen untersucht wird. In einigen F\u00e4llen kann auch ein Provokationstest durchgef\u00fchrt werden, bei dem das Allergen kontrolliert in den K\u00f6rper eingebracht wird, um eine allergische Reaktion auszul\u00f6sen und zu beobachten. Es ist wichtig, dass diese Untersuchungen von einem Facharzt f\u00fcr Allergologie durchgef\u00fchrt werden, um eine genaue Diagnose zu stellen und eine geeignete Behandlung zu empfehlen.""",0.0,1.0,0.0
3,"""Wie untersuche ich eine Wespenstichallergie?""","""Um eine Wespenstichallergie zu untersuchen, k\u00f6nnen verschiedene diagnostische Verfahren eingesetzt werden. Eine M\u00f6glichkeit ist der sogenannte Hauttest, bei dem eine kleine Menge des Allergens auf die Haut aufgetragen wird und anschlie\u00dfend die Reaktion beobachtet wird. Eine andere Methode ist der Bluttest, bei dem das Blut auf spezifische Antik\u00f6rper gegen das Wespenallergen untersucht wird. In einigen F\u00e4llen kann auch ein Provokationstest durchgef\u00fchrt werden, bei dem das Allergen kontrolliert in den K\u00f6rper eingebracht wird, um eine allergische Reaktion auszul\u00f6sen und zu beobachten. Es ist wichtig, dass diese Untersuchungen von einem Facharzt f\u00fcr Allergologie durchgef\u00fchrt werden, um eine genaue Diagnose zu stellen und eine geeignete Behandlung zu empfehlen.""",0.0,1.0,0.0
4,"""Was sind die ersten Tests um AIDS zu diagnostizieren?""","""Die ersten Tests zur Diagnose von AIDS sind der kombinierte HIV-Antigen-Antik\u00f6rpersuchtest (HIV-Screening-Test der 4. Generation), der HIV-spezifische Antik\u00f6rper und das HIV-spezifische p24-Antigen nachweist. Bei Kindern \u00fcber 24 Monaten ist dies die bevorzugte Methode. Bei akuter HIV-Infektion kann die Sensitivit\u00e4t di

In [34]:
tru.get_leaderboard(app_ids=[])
tru.run_dashboard()

Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://localhost:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>